Logs

- [2022/02/01]   
  Scraping `antaranews.com`. Using static web scraping responds 500.  
  Using dynamic web scraping.
  Using double quotes in query more than words makes a trouble for the next page.

  Some useful references:
  - `https://stackoverflow.com/questions/35747252/navigate-between-pages-with-selenium-java`

  Clicking the WebElement does not work to get a next page.  
  Then `browser.get` should be used because the `a href` of the next page can be retrieved.   

  `antaranews.com`'s search engine does not work like it supposed to do when the range of 
  date is filtered.   
  Then it scraped first all the previous date from the current date.  
  After that, it selects specific range of date

In [390]:
import requests
import xlsxwriter
import time
import datetime
import pickle
import re

import bs4
from bs4 import BeautifulSoup

import selenium
from seleniumwire import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [267]:
print(f"selenium.__version__: {selenium.__version__}")
bs4.__version__

selenium.__version__: 3.141.0


'4.9.3'

## Get a specific type of news

In [281]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--kiosk-printing')
#chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')

browser = webdriver.Chrome(
  executable_path="/home/henokh/Downloads/chromedriver_linux64/chromedriver", 
  options=chrome_options)

print("Go to the URL")
base_site = "https://antaranews.com/search/"
query = 'unjuk rasa'

page = 1
start_date = "01-01-2021"
end_date = "31-12-2021"
query_full = "-".join(query.split()) + f"/{page}/" 
date_range = f"&startDate={start_date}&endDate={end_date}"
query_loc = "&inTitle=1&inContent=1"
print(f"url: {base_site + query_full + date_range + query_loc}")

browser.get(base_site + query_full + date_range + query_loc)

Go to the URL
url: https://antaranews.com/search/unjuk-rasa/1/&startDate=01-01-2021&endDate=31-12-2021&inTitle=1&inContent=1


In [282]:
print("Get all URLs (with its date and time) in each page")
elem = browser.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

soup_page = BeautifulSoup(source_code, parser='lxml')

[[url_.find("header").find('span').get_text().strip(), 
  url_.find('a')["href"]] 
    for url_ in soup_page.find_all('article', attrs={'class': 'simple-post simple-big clearfix'})]

Get all URLs (with its date and time) in each page


[['30 Januari 2022 11:32',
  'https://www.antaranews.com/berita/2673913/merusak-alam-tambang-emas-ilegal-gunung-prabu-lombok-tengah-ditutup'],
 ['28 Januari 2022 21:56',
  'https://www.antaranews.com/berita/2672261/polri-siapkan-skenario-pengamanan-sukseskan-forum-gpdrr-2022-bali'],
 ['28 Januari 2022 18:57',
  'https://www.antaranews.com/berita/2671737/lemkapi-anarkis-saat-demo-harus-ditindak-tegas'],
 ['27 Januari 2022 19:58',
  'https://www.antaranews.com/berita/2669629/polisi-sebut-16-dari-725-anggota-gmbi-yang-diamankan-positif-narkoba'],
 ['27 Januari 2022 18:25',
  'https://www.antaranews.com/berita/2669361/ratusan-anggota-ormas-yang-gelar-aksi-di-depan-polda-jabar-diamankan'],
 ['27 Januari 2022 17:37',
  'https://www.antaranews.com/berita/2669169/bin-diy-memfasilitasi-deklarasi-damai-mahasiswa-maluku-di-yogyakarta'],
 ['26 Januari 2022 20:49',
  'https://www.antaranews.com/berita/2667205/pemindahan-ibu-kota-dinilai-akan-berdampak-pada-ekonomi-jakarta'],
 ['26 Januari 2022 19:3

In [305]:
month_dict = {
  "Januari": "01", "Februari": "02", "Maret": "03",
  "April": "04", "Mei": "05", "Juni": "06",
  "Juli": "07", "Agustus": "08", "September": "09",
  "Oktober": "10", "November": "11", "Desember": "12"}
t1 = all_urls[-1][0]
t1 = t1.split()[:-1]
t1[1] = month_dict[t1[1]]
t1 = "/".join(t1)
t1 = datetime.datetime.strptime(t1, "%d/%m/%Y")
t1

datetime.datetime(2018, 12, 18, 0, 0)

In [306]:
t_start = datetime.datetime(2019, 1, 1)

t1 < t_start

True

In [312]:
t1.strftime("%Y/%m/%d")

'2018/12/18'

In [313]:
t1.year

2018

## Iterate over all pages with condition on specific range of the date

In [219]:
page = 10

parent_ul = browser.find_elements_by_xpath("//ul[@class='pagination pagination-sm']")
element_list = parent_ul[0].find_elements_by_tag_name("li")
element_text = [element.text for element in element_list]

idx_next_element = [i for i, text in enumerate(element_text) if text == str(page)]

if len(idx_next_element) == 0:
  next_element = element_list[-2]
else:
  next_element = element_list[idx_next_element[0]]

print(f"element_text: {element_text}")
print(f"idx_next_element: {idx_next_element}")
print(f"next_element: {next_element}")
print(f"next_element.text: {next_element.text}")

element_text: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '»']
idx_next_element: [9]
next_element: <selenium.webdriver.remote.webelement.WebElement (session="5f7567f44025a170e2a9602fa4e2fcf8", element="e1e62340-957a-4095-8b69-48537acfb975")>
next_element.text: 10


In [226]:
next_element.find_element_by_xpath(".//a").get_attribute("href")

'https://www.antaranews.com/search/unjuk-rasa/10?inTitle=1&inContent=1&startDate=01-01-2021&endDate=31-12-2021'

23 minutes to scrap a single keywords for data from 2019 to now

In [299]:
month_dict = {
  "Januari": "01", "Februari": "02", "Maret": "03",
  "April": "04", "Mei": "05", "Juni": "06",
  "Juli": "07", "Agustus": "08", "September": "09",
  "Oktober": "10", "November": "11", "Desember": "12"}
t_start = datetime.datetime(2019, 1, 1)


# -- create webdriver object
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--kiosk-printing')
#chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')

browser = webdriver.Chrome(
  executable_path="/home/henokh/Downloads/chromedriver_linux64/chromedriver", 
  options=chrome_options)


# -- get a webpage for a given query
print("Go to the URL")
base_site = "https://antaranews.com/search/"
query = 'unjuk rasa'

page = 1
start_date = "01-01-2019"
end_date = "31-12-2021"
query_full = "-".join(query.split()) + f"/{page}/" 
date_range = f"&startDate={start_date}&endDate={end_date}"
query_loc = "&inTitle=1&inContent=1"
print(f"url: {base_site + query_full + date_range + query_loc}")

browser.get(base_site + query_full + date_range + query_loc)

# -- get all URLs in first page
print(f"Get all URLs (with its date and time) in page: {page:03d}")
elem = browser.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

soup_page = BeautifulSoup(source_code, parser='lxml')

all_urls = [[url_.find("header").find('span').get_text().strip(), 
  url_.find('a')["href"]] 
    for url_ in soup_page.find_all('article', attrs={'class': 'simple-post simple-big clearfix'})]


# -- iterate to the next page
list_of_urls_dict = {page: all_urls}

while True:
  page += 1

  parent_ul = browser.find_elements_by_xpath("//ul[@class='pagination pagination-sm']")
  #print(f"   parent_ul: {parent_ul}")
  element_list = parent_ul[0].find_elements_by_tag_name("li") 
  element_text = [element.text for element in element_list]
  
  next_element = element_list[-1]
  next_group_page_link = next_element.find_elements_by_xpath(".//a")[0].get_attribute("href")
  print(f"   next_group_page_link: {next_group_page_link}")
  
  while True:
    try:
      browser.get(next_group_page_link)
      browser.implicitly_wait(2)   # <--- it has to be adjusted depending on your internet connection speed
      break
    except WebDriverException:
      continue

  print(f"Get all URLs (with its date and time) in page: {page:03d}")
  element_each_page = browser.find_element_by_xpath("//*")
  source_code = element_each_page.get_attribute("outerHTML")

  soup_page = BeautifulSoup(source_code, parser='lxml')

  all_urls = [[url_.find("header").find('span').get_text().strip(), 
  url_.find('a')["href"]] 
    for url_ in soup_page.find_all('article', attrs={'class': 'simple-post simple-big clearfix'})]
  list_of_urls_dict[page] = all_urls


  t_last_url = all_urls[-1][0]
  t_last_url = t_last_url .split()[:-1]
  t_last_url[1] = month_dict[t_last_url[1]]
  t_last_url = "/".join(t_last_url)
  t_last_url = datetime.datetime.strptime(t_last_url, "%d/%m/%Y")

  if t_last_url < t_start:
    break

Go to the URL
url: https://antaranews.com/search/unjuk-rasa/1/&startDate=01-01-2019&endDate=31-12-2021&inTitle=1&inContent=1
Get all URLs (with its date and time) in page: 001
   next_group_page_link: https://www.antaranews.com/search/unjuk-rasa/2?inTitle=1&inContent=1&startDate=01-01-2019&endDate=31-12-2021
Get all URLs (with its date and time) in page: 002
   next_group_page_link: https://www.antaranews.com/search/unjuk-rasa/3?inTitle=1&inContent=1&startDate=01-01-2019&endDate=31-12-2021
Get all URLs (with its date and time) in page: 003
   next_group_page_link: https://www.antaranews.com/search/unjuk-rasa/4?inTitle=1&inContent=1&startDate=01-01-2019&endDate=31-12-2021
Get all URLs (with its date and time) in page: 004
   next_group_page_link: https://www.antaranews.com/search/unjuk-rasa/5?inTitle=1&inContent=1&startDate=01-01-2019&endDate=31-12-2021
Get all URLs (with its date and time) in page: 005
   next_group_page_link: https://www.antaranews.com/search/unjuk-rasa/6?inTitle=1&in

In [300]:
list_of_urls_dict

{1: [['30 Januari 2022 11:32',
   'https://www.antaranews.com/berita/2673913/merusak-alam-tambang-emas-ilegal-gunung-prabu-lombok-tengah-ditutup'],
  ['28 Januari 2022 21:56',
   'https://www.antaranews.com/berita/2672261/polri-siapkan-skenario-pengamanan-sukseskan-forum-gpdrr-2022-bali'],
  ['28 Januari 2022 18:57',
   'https://www.antaranews.com/berita/2671737/lemkapi-anarkis-saat-demo-harus-ditindak-tegas'],
  ['27 Januari 2022 19:58',
   'https://www.antaranews.com/berita/2669629/polisi-sebut-16-dari-725-anggota-gmbi-yang-diamankan-positif-narkoba'],
  ['27 Januari 2022 18:25',
   'https://www.antaranews.com/berita/2669361/ratusan-anggota-ormas-yang-gelar-aksi-di-depan-polda-jabar-diamankan'],
  ['27 Januari 2022 17:37',
   'https://www.antaranews.com/berita/2669169/bin-diy-memfasilitasi-deklarasi-damai-mahasiswa-maluku-di-yogyakarta'],
  ['26 Januari 2022 20:49',
   'https://www.antaranews.com/berita/2667205/pemindahan-ibu-kota-dinilai-akan-berdampak-pada-ekonomi-jakarta'],
  ['26

In [302]:
# Save data
# with open(f"[{query}]-antaranews_2019_2022.pkl", "wb") as data:
#   pickle.dump(list_of_urls_dict, data)

In [303]:
# Load data
with open(f"[{query}]-antaranews_2019_2022.pkl", "rb") as data:
  loaded_list_of_urls_dict =  pickle.load(data)

## Parsing only specific year and scraping the news

In [314]:
selected_year = 2021

month_dict = {
  "Januari": "01", "Februari": "02", "Maret": "03",
  "April": "04", "Mei": "05", "Juni": "06",
  "Juli": "07", "Agustus": "08", "September": "09",
  "Oktober": "10", "November": "11", "Desember": "12"}

list_of_urls_selected_year = []
for k, v in loaded_list_of_urls_dict.items():
  for dt, news_url in v:
    t_news = dt.split()[:-1]
    t_news[1] = month_dict[t_news[1]]
    t_news = "/".join(t_news)
    t_news = datetime.datetime.strptime(t_news, "%d/%m/%Y")

    dt = t_news.strftime("%Y/%m/%d")
    if t_news.year == 2021:
      list_of_urls_selected_year.append((dt, news_url))

    

In [315]:
list_of_urls_selected_year

[('2021/12/31',
  'https://www.antaranews.com/berita/2617777/polrestro-jakarta-selatan-terima-1504-laporan-sepanjang-tahun-2021'),
 ('2021/12/31',
  'https://www.antaranews.com/foto/2617217/unjuk-rasa-warga-menentang-kudeta-militer-di-sudan'),
 ('2021/12/29',
  'https://www.antaranews.com/berita/2613153/anggota-dpr-bersedia-jadi-mediator-gubernur-banten-buruh'),
 ('2021/12/29',
  'https://www.antaranews.com/berita/2613097/prahara-buruh-banten-di-penghujung-2021'),
 ('2021/12/29',
  'https://www.antaranews.com/berita/2612997/ketua-dprd-banten-berharap-buruh-dan-gubernur-saling-memaafkan'),
 ('2021/12/28',
  'https://www.antaranews.com/berita/2611653/polda-banten-menangguhkan-penahanan-buruh-yang-jadi-tersangka'),
 ('2021/12/27',
  'https://www.antaranews.com/berita/2609509/ipw-kasus-buruh-di-banten-perlu-pendekatan-keadilan-restoratif'),
 ('2021/12/27',
  'https://www.antaranews.com/berita/2609233/aji-palu-catat-lima-kasus-kekerasan-terhadap-jurnalis-di-tahun-2021'),
 ('2021/12/26',
  '

### Test for one url entry

In [319]:
nth_url_entry = 0

first_entry_response = requests.get(list_of_urls_selected_year[0][1])
print(first_entry_response.status_code, first_entry_response.url)

200 https://www.antaranews.com/berita/2617777/polrestro-jakarta-selatan-terima-1504-laporan-sepanjang-tahun-2021


In [393]:
first_entry_pages = ""
first_entry_html_page = first_entry_response.content
first_entry_soup_page = BeautifulSoup(first_entry_html_page, parser="lxml")

first_entry_content = first_entry_soup_page.find("div", attrs={"class": "post-content clearfix"})
#print(f"first_entry_content: {first_entry_content}")

# remove <div class="quote-old">
for tag in first_entry_content.find_all("div", attrs={"class": "quote_old"}):
  #print(tag)
  tag.replaceWith("")

# remove <span class="baca-juga">
for tag in first_entry_content.find_all("span", attrs={"class": "baca-juga"}):
  tag.replaceWith("")

# remove <p class="text-muted small mt10">
for tag in first_entry_content.find_all("p", attrs={"class": "text-muted small mt10"}):
  tag.replaceWith("")

# remove <script>
for tag in first_entry_content.find_all("script", attrs={"async": ""}):
  tag.replaceWith("")

# remove <ins>
for tag in first_entry_content.find_all("ins", attrs={"class": "adsbygoogle"}):
  tag.replaceWith("")

# remove tag <em> not the content
for tag in first_entry_content.find_all("em"):
  tag.replaceWith(tag.get_text())

# remove tag <br/>
for tag in first_entry_content.find_all("br"):
  tag.replaceWith("")


first_entry_pars = first_entry_content.prettify(formatter="html")
#first_entry_pars = first_entry_content.get_text()

# remove \n, \r, and &nbsp;
first_entry_pars = first_entry_pars.replace("\n", "")
first_entry_pars = first_entry_pars.replace("\r", "")
first_entry_pars = first_entry_pars.replace("&nbsp;", "")

# remove HTML tag  (it removes after it output into string)
CLEANR = re.compile("<.*?>")
first_entry_pars = re.sub(CLEANR, "", first_entry_pars).strip()
first_entry_pars

'Jakarta (ANTARA) - Polres Metro Jakarta Selatan menerima sebanyak 1.504 laporan dari masyarakat terkait berbagai kasus sepanjang tahun 2021. Kapolres Metro Jakarta Selatan Komisaris Besar Polisi Azis Andriansyah mengatakan dari ribuan laporan tersebut, pihaknya telah menyelesaikan 1.444 laporan. "Kita bisa menyelesaikan 1.444 kasus atau crime clear 96 persen dari jumlah 1.504, nah jumlah laporan polisi ini berkurang 30 persen dari tahun sebelumnya, tahun sebelumnya sekitar 1.800-n tahun ini 1.504," kata Azis di Jakarta, Jumat. Azis menyebutkan dari ribuan laporan masuk, pihaknya mencatat lima tren kasus kejahatan yang terjadi di kawasan Jakarta Selatan, yakni kasus narkotika, pencurian dengan pemberatan, pencurian kendaraan bermotor, dan kasus kejahatan terhadap perempuan dan anak. Kemudian, kasus berkaitan kenakalan anak dan remaja, dan terakhir kasus pembunuhan. "Press rilis kita laksanakan supaya kita bisa introspeksi apa saja yang sudah kita lakukan dan apa saja yang perlu kita la

### Get all texts from all urls 

In [399]:
def get_text_from_url(valid_url):
  response = requests.get(valid_url)
  print(f"   {response.status_code} {response.url}")

  pages = ""
  html_page = response.content
  soup_page = BeautifulSoup(html_page, parser="lxml")

  content = soup_page.find("div", attrs={"class": "post-content clearfix"})
  #print(f"content: {content}")

  # remove <div class="quote-old">
  for tag in content.find_all("div", attrs={"class": "quote_old"}):
    #print(tag)
    tag.replaceWith("")

  # remove <span class="baca-juga">
  for tag in content.find_all("span", attrs={"class": "baca-juga"}):
    tag.replaceWith("")

  # remove <p class="text-muted small mt10">
  for tag in content.find_all("p", attrs={"class": "text-muted small mt10"}):
    tag.replaceWith("")

  # remove <script>
  for tag in content.find_all("script", attrs={"async": ""}):
    tag.replaceWith("")

  # remove <ins>
  for tag in content.find_all("ins", attrs={"class": "adsbygoogle"}):
    tag.replaceWith("")

  # remove tag <em> not the content
  for tag in content.find_all("em"):
    tag.replaceWith(tag.get_text())

  # remove tag <br/>
  for tag in content.find_all("br"):
    tag.replaceWith("")


  pars = content.prettify(formatter="html")
  #pars = content.get_text()

  # remove \n, \r, and &nbsp;
  pars = pars.replace("\n", "")
  pars = pars.replace("\r", "")
  pars = pars.replace("&nbsp;", "")

  # remove HTML tag  (it removes after it output into string)
  CLEANR = re.compile("<.*?>")
  pars = re.sub(CLEANR, "", pars).strip()
  
  return pars

In [401]:
news_data = []
for dt, valid_url in list_of_urls_selected_year:
  
  # omitting news that only contains photo (no text
  if "/foto/" in valid_url:
    continue
  
  print(dt)
  news_data.append((dt, get_text_from_url(valid_url)))

2021/12/31
   200 https://www.antaranews.com/berita/2617777/polrestro-jakarta-selatan-terima-1504-laporan-sepanjang-tahun-2021
2021/12/29
   200 https://www.antaranews.com/berita/2613153/anggota-dpr-bersedia-jadi-mediator-gubernur-banten-buruh
2021/12/29
   200 https://www.antaranews.com/berita/2613097/prahara-buruh-banten-di-penghujung-2021
2021/12/29
   200 https://www.antaranews.com/berita/2612997/ketua-dprd-banten-berharap-buruh-dan-gubernur-saling-memaafkan
2021/12/28
   200 https://www.antaranews.com/berita/2611653/polda-banten-menangguhkan-penahanan-buruh-yang-jadi-tersangka
2021/12/27
   200 https://www.antaranews.com/berita/2609509/ipw-kasus-buruh-di-banten-perlu-pendekatan-keadilan-restoratif
2021/12/27
   200 https://www.antaranews.com/berita/2609233/aji-palu-catat-lima-kasus-kekerasan-terhadap-jurnalis-di-tahun-2021
2021/12/26
   200 https://www.antaranews.com/berita/2608133/p4mu-adukan-oknum-spsi-jatim-ke-komnas-ham
2021/12/25
   200 https://www.antaranews.com/berita/26070

## Save the results

In [407]:
with xlsxwriter.Workbook(f"{selected_year}-[{query}]-antaranews.xlsx") as workbook:
    worksheet = workbook.add_worksheet()

    for row_num, (dt, news_text) in enumerate(news_data):
        worksheet.write(row_num, 0, dt)
        worksheet.write(row_num, 1, news_text)